In [30]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [31]:
# # 구글 드라이브 마운팅
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [32]:
# # 경로 지정
# path = '/content/drive/My Drive/projects/Nalssi' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
# os.chdir(path)

## 데이터 로드

In [33]:
df_raw = pd.read_csv('smart_farm_y0.csv').drop('Unnamed: 0', axis=1)
df_raw

,insolation,out_tmperature,out_humidity,wind_speed,wind_direction,in_tmperature,in_humidity,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site,ventilation_temperature_control,heating_temperature_set_up,rainfall,heat_supply
0,0.002248,0.555133,0.701149,0.078788,0.907042,0.421941,0.514196,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
1,0.002248,0.551331,0.701149,0.000000,0.771831,0.459916,0.446372,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333,0.5,0.020166,0.0
2,0.002473,0.543726,0.701149,0.000000,0.752113,0.426160,0.482650,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
3,0.002698,0.539924,0.701149,0.000000,0.752113,0.426160,0.490536,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
4,0.002811,0.539924,0.701149,0.000000,0.752113,0.430380,0.493691,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40378,0.002811,0.570342,0.425287,0.000000,0.146479,0.434599,0.672240,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
40379,0.002698,0.566540,0.448276,0.000000,0.146479,0.430380,0.665300,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0
40380,0.002923,0.555133,0.448276,0.000000,0.146479,0.453586,0.629338,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333,0.5,0.020166,0.0
40381,0.003035,0.551331,0.448276,0.024242,0.146479,0.428270,0.663013,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.5,0.020166,0.0


## 훈련을 위한 데이터 분류

In [49]:
# 데이터프레임 x,y,z 나누기
def df_split(data):
    global x_raw, y_raw, z_raw, y_li
    x_li = ['insolation', 'out_tmperature', 'out_humidity', 'wind_speed',
            'wind_direction', 'in_tmperature', 'in_humidity', 'rainfall']
    y_li = ['shield_light_h', 'shield_tmperature_h', 'shield_energy_h',
            'shield_energy_v', 'exhaust_fan', 'ceiling', 'floating_fan',
            'fan_coil_b_site']
    z_li = ['heat_supply']
    x_raw = data[x_li]
    y_raw = data[y_li]
    z_raw = data[z_li]

In [50]:
df_split(df_raw)

In [51]:
# output 나누기
def format_output(data):
    global y_li
    for i , y in enumerate(y_li):
        globals()['o{}'.format(i)] = data[y]
        # return globals()['o{}'.format(i)]
    return o0, o1, o2, o3, o4, o5, o6, o7

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=.8)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=.5)

In [53]:
x_raw

,insolation,out_tmperature,out_humidity,wind_speed,wind_direction,in_tmperature,in_humidity,rainfall
0,0.002248,0.555133,0.701149,0.078788,0.907042,0.421941,0.514196,0.020166
1,0.002248,0.551331,0.701149,0.000000,0.771831,0.459916,0.446372,0.020166
2,0.002473,0.543726,0.701149,0.000000,0.752113,0.426160,0.482650,0.020166
3,0.002698,0.539924,0.701149,0.000000,0.752113,0.426160,0.490536,0.020166
4,0.002811,0.539924,0.701149,0.000000,0.752113,0.430380,0.493691,0.020166
...,...,...,...,...,...,...,...,...
40378,0.002811,0.570342,0.425287,0.000000,0.146479,0.434599,0.672240,0.020166
40379,0.002698,0.566540,0.448276,0.000000,0.146479,0.430380,0.665300,0.020166
40380,0.002923,0.555133,0.448276,0.000000,0.146479,0.453586,0.629338,0.020166
40381,0.003035,0.551331,0.448276,0.024242,0.146479,0.428270,0.663013,0.020166


## 모델 생성

In [54]:
# 배치 사이즈 20

input_layer = Input(shape=(len(x_train.columns), ))
first_dense = Dense(units=200, activation='relu')(input_layer)
drop1 = Dropout(.3)(first_dense)
second_dense = Dense(units=100, activation='relu')(drop1)
drop2 = Dropout(.3)(second_dense)
third_dense = Dense(units=50, activation='relu')(second_dense)
fourth_dense = Dense(units=20, activation='relu')(third_dense)

output_layer = []
for i in range(1, 9):
        globals()['output{}'.format(i)] = Dense(units=1, name='output{}'.format(i))(fourth_dense)
        output_layer.append(globals()['output{}'.format(i)])

In [55]:
model = Model(inputs=input_layer, outputs=output_layer)

## 모델 훈련

In [56]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss = 'mse', metrics='accuracy')

In [57]:
history = model.fit(x = x_train, 
                    y = format_output(y_train), 
                    batch_size=20, epochs=10, 
                    validation_data=(x_val, format_output(y_val)), 
                    verbose=1)

Epoch 1/10
404/404 [==============================] - 3s 5ms/step - loss: 0.6928 - output1_loss: 0.0617 - output2_loss: 0.0946 - output3_loss: 0.0838 - output4_loss: 0.0701 - output5_loss: 0.0086 - output6_loss: 0.0136 - output7_loss: 0.1333 - output8_loss: 0.2271 - output1_accuracy: 0.9142 - output2_accuracy: 0.8609 - output3_accuracy: 0.8370 - output4_accuracy: 0.9154 - output5_accuracy: 1.0000 - output6_accuracy: 0.8390 - output7_accuracy: 0.8288 - output8_accuracy: 0.6607 - val_loss: 0.3968 - val_output1_loss: 0.0179 - val_output2_loss: 0.0416 - val_output3_loss: 0.0463 - val_output4_loss: 0.0298 - val_output5_loss: 0.0012 - val_output6_loss: 0.0052 - val_output7_loss: 0.0703 - val_output8_loss: 0.1845 - val_output1_accuracy: 0.9598 - val_output2_accuracy: 0.9245 - val_output3_accuracy: 0.8827 - val_output4_accuracy: 0.9552 - val_output5_accuracy: 1.0000 - val_output6_accuracy: 0.8425 - val_output7_accuracy: 0.9200 - val_output8_accuracy: 0.7141
Epoch 2/10
404/404 [================

404/404 [==============================] - 2s 4ms/step - loss: 0.3009 - output1_loss: 0.0107 - output2_loss: 0.0253 - output3_loss: 0.0339 - output4_loss: 0.0214 - output5_loss: 3.8620e-04 - output6_loss: 0.0022 - output7_loss: 0.0644 - output8_loss: 0.1425 - output1_accuracy: 0.9673 - output2_accuracy: 0.9399 - output3_accuracy: 0.8887 - output4_accuracy: 0.9609 - output5_accuracy: 1.0000 - output6_accuracy: 0.8390 - output7_accuracy: 0.9229 - output8_accuracy: 0.7828 - val_loss: 0.2875 - val_output1_loss: 0.0123 - val_output2_loss: 0.0228 - val_output3_loss: 0.0285 - val_output4_loss: 0.0201 - val_output5_loss: 2.1970e-04 - val_output6_loss: 0.0021 - val_output7_loss: 0.0641 - val_output8_loss: 0.1373 - val_output1_accuracy: 0.9581 - val_output2_accuracy: 0.9404 - val_output3_accuracy: 0.8997 - val_output4_accuracy: 0.9631 - val_output5_accuracy: 1.0000 - val_output6_accuracy: 0.8425 - val_output7_accuracy: 0.9218 - val_output8_accuracy: 0.7973
Epoch 10/10
404/404 [==================

## 모델 평가

In [112]:
score = model.evaluate(x_test, format_output(y_test), verbose=1)

9311/9311 [==============================] - 50s 5ms/step - loss: 0.2152 - output1_loss: 0.0015 - output2_loss: 0.0047 - output3_loss: 0.0110 - output4_loss: 0.0076 - output5_loss: 0.0534 - output6_loss: 0.0044 - output7_loss: 0.0587 - output8_loss: 0.0740 - output1_accuracy: 0.9730 - output2_accuracy: 0.9641 - output3_accuracy: 0.9283 - output4_accuracy: 0.9779 - output5_accuracy: 0.9214 - output6_accuracy: 0.7286 - output7_accuracy: 0.9219 - output8_accuracy: 0.8960


In [113]:
import matplotlib.pyplot as plt

In [119]:
def plot_loss_acc(history, epoch):

    hist = pd.DataFrame(history.history)

    plt.plot(range(1, epoch + 1), hist.iloc[:, 0], label='Training')
    plt.plot(range(1, epoch + 1), hist.iloc[:, 17], label='Validation')
    plt.legend(loc='best')
    plt.title('Loss')
    
    for i in range(1,9):
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        axes[0].plot(range(1, epoch + 1), hist.iloc[:, 0+i], label='Training')
        axes[0].plot(range(1, epoch + 1), hist.iloc[:, 17+i], label='Validation')
        axes[0].legend(loc='best')
        if i>0:
            axes[0].set_title(f'y{i}_Loss')
        

        try : 
            axes[1].plot(range(1, epoch + 1), hist.iloc[:, 8+i], label='Training')
            axes[1].plot(range(1, epoch + 1), hist.iloc[:, 25+i], label='Validation')
            axes[1].legend(loc='best')
            axes[1].set_title(f'y{i}_Accuracy')
        except IndexError as e : 
            pass
        plt.show()

In [ ]:
plot_loss_acc(history, 10)

## 모델에 열공급량 0.1~1.0까지의 X값 넣어서 Ys 값 예측해보기

### 열공급량 0.1~1.0 데이터 로드

In [114]:
# for i in range(1, 11):
#     globals()['df_y{}'.format(i)] = pd.read_csv('smart_farm_y{}.csv'.format(i)).drop('Unnamed: 0', axis=1)

In [26]:
# Ys 값 예측 함수 정의
def y_pred_cal(data):
    global x_raw, y_raw
    y_pred = model.predict(x_raw)
    m, n, r = np.array(y_pred).shape
    out_arr = np.column_stack((np.repeat(np.arange(r), m), np.array(y_pred).reshape(r*m,-1)))
    out_df = pd.DataFrame(out_arr).T
    out_df.drop([0], inplace = True)
    out_df.columns = y_raw.columns
    out_df.reset_index(inplace = True)
    global y_pred_df
    y_pred_df = out_df.drop('index', axis = 1)
    return y_pred_df

# 차이값 확인 함수 정의
def diff_cal(y_raw):
    global y_pred_df
    if y_pred_df.shape == y_raw.shape:
        diff_df = y_pred_df.sub(y_raw)
        return diff_df
    else:
        return "The shapes of data are not the same!"

### 함수사용해서 Ys값 및 차이값 확인

In [116]:
y_pred_cal(df_y1)

,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site
0,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,0.999477,0.827087
1,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,0.999477,0.827087
2,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,0.999477,0.827087
3,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,0.999477,0.827087
4,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,0.999477,0.827087
...,...,...,...,...,...,...,...,...
744866,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,1.020242,0.908892
744867,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,1.020242,0.908892
744868,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,1.020242,0.908892
744869,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,1.020242,0.908892


In [117]:
diff_cal(y_raw)

,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site
0,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,-0.000523,-0.172913
1,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,-0.000523,-0.172913
2,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,-0.000523,-0.172913
3,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,-0.000523,-0.172913
4,0.004579,0.011056,-0.009861,0.011181,-0.023570,0.002280,-0.000523,-0.172913
...,...,...,...,...,...,...,...,...
744866,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,0.020242,-0.091108
744867,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,0.020242,-0.091108
744868,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,0.020242,-0.091108
744869,0.008162,0.045448,-0.011241,0.001254,-0.016647,0.006591,0.020242,-0.091108


In [118]:
diff_cal(y_raw).describe()

,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site
count,744871.000000,744871.000000,744871.000000,744871.000000,744871.000000,744871.000000,744871.000000,744871.000000
mean,-0.003863,-0.004394,-0.008772,0.001006,0.003405,-0.000279,-0.004757,-0.045149
std,0.038426,0.069064,0.104354,0.087131,0.230894,0.066281,0.240969,0.269033
min,-0.915894,-0.986278,-0.968132,-0.974215,-1.040678,-0.531367,-0.727501,-1.100954
25%,-0.011173,-0.012833,-0.019952,-0.007622,-0.016253,-0.012699,-0.115689,-0.064271
50%,-0.004212,-0.004339,-0.008624,0.000276,-0.000138,0.001156,-0.008066,-0.004758
75%,0.002833,0.004967,0.003355,0.008823,0.027794,0.013519,0.022721,0.036775
max,0.880992,0.965215,0.863621,1.013857,1.011016,0.579761,1.050383,0.979560


## 예측값 넣은 df 만들기

In [25]:
# 0.1~1.0사의 값 가져오기
df_y = pd.read_csv('smart_farm_y_test.csv').drop('Unnamed: 0', axis=1)
df_y

,insolation,out_tmperature,out_humidity,wind_speed,wind_direction,in_tmperature,in_humidity,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site,ventilation_temperature_control,heating_temperature_set_up,rainfall,heat_supply
0,0.001911,0.433460,0.827586,0.054545,0.726761,0.434599,0.424290,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.500000,0.020166,0.1
1,0.001799,0.437262,0.816092,0.000000,0.740845,0.447257,0.403785,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.500000,0.020166,0.1
2,0.001686,0.437262,0.816092,0.024242,0.738028,0.447257,0.411672,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.500000,0.020166,0.1
3,0.001911,0.441065,0.804598,0.000000,0.738028,0.459916,0.397476,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333,0.500000,0.020166,0.1
4,0.002024,0.441065,0.804598,0.000000,0.738028,0.455696,0.394322,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333333,0.500000,0.020166,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80133,0.002586,0.380228,0.252874,0.078788,0.873239,0.198312,0.840694,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333333,0.016667,0.020166,0.9
80134,0.001799,0.733840,0.218391,0.024242,0.791549,0.451477,0.806309,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.550000,0.020166,0.9
80135,0.001349,0.730038,0.218391,0.109091,0.588732,0.449367,0.799369,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.550000,0.020166,0.9
80136,0.001461,0.730038,0.218391,0.078788,0.650704,0.447257,0.820347,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.533333,0.020166,0.9


In [59]:
x_li = ['insolation', 'out_tmperature', 'out_humidity', 'wind_speed',
        'wind_direction', 'in_tmperature', 'in_humidity', 'rainfall']
y_li = ['shield_light_h', 'shield_tmperature_h', 'shield_energy_h',
        'shield_energy_v', 'exhaust_fan', 'ceiling', 'floating_fan',
        'fan_coil_b_site']
z_li = ['heat_supply']
df2_x_raw = df_y[x_li]
df2_y_raw = df_y[y_li]

In [81]:
# df2_x_raw.shape
# df2_x_raw

In [67]:
a = model.predict(df2_x_raw)

In [69]:
m, n, r = np.array(a).shape
print(m, n, r)

8 80138 1


In [77]:
out_arr = np.column_stack((np.repeat(np.arange(r), m), np.array(a).reshape(r*m,-1)))
# out_arr

In [79]:
out_df = pd.DataFrame(out_arr).T
out_df
out_df.drop([0], inplace = True)
out_df.columns = y_raw.columns
out_df.reset_index(inplace = True)
y_pred_df = out_df.drop('index', axis = 1)
y_pred_df

,shield_light_h,shield_tmperature_h,shield_energy_h,shield_energy_v,exhaust_fan,ceiling,floating_fan,fan_coil_b_site
0,0.004403,0.003184,-0.009332,0.024189,0.005625,-0.014604,0.969542,0.361670
1,0.006043,0.002858,-0.012593,0.025955,0.006019,-0.016605,0.980418,0.350026
2,0.006924,0.001455,-0.012319,0.026261,0.006175,-0.016482,0.975614,0.351699
3,0.008562,0.001096,-0.016948,0.027894,0.004461,-0.015980,0.981625,0.350604
4,0.007724,0.001386,-0.015833,0.027174,0.004637,-0.016097,0.983651,0.353274
...,...,...,...,...,...,...,...,...
80133,-0.007398,-0.045212,-0.001876,0.042642,-0.010455,0.028698,1.018001,0.233532
80134,0.111542,1.010720,0.025707,-0.048665,-0.017880,0.002723,0.960472,0.073547
80135,0.062339,0.967617,0.030119,-0.070909,-0.023549,-0.001962,0.947668,0.072423
80136,0.061123,0.982701,0.033540,-0.072471,-0.026541,0.000453,0.955821,0.079399


In [ ]:
out_df.drop([0], inplace = True)
out_df.columns = y_raw.columns
out_df.reset_index(inplace = True)
global y_pred_df
y_pred_df = out_df.drop('index', axis = 1)

In [86]:
# 예측y값과 원본 x 합치기
df = pd.concat([df2_x_raw, y_pred_df], axis = 1)
df.to_csv('smart_farm_final_x.csv')

In [47]:
df_y.iloc[:,:7]

,insolation,out_tmperature,out_humidity,wind_speed,wind_direction,in_tmperature,in_humidity
0,0.001911,0.433460,0.827586,0.054545,0.726761,0.434599,0.424290
1,0.001799,0.437262,0.816092,0.000000,0.740845,0.447257,0.403785
2,0.001686,0.437262,0.816092,0.024242,0.738028,0.447257,0.411672
3,0.001911,0.441065,0.804598,0.000000,0.738028,0.459916,0.397476
4,0.002024,0.441065,0.804598,0.000000,0.738028,0.455696,0.394322
...,...,...,...,...,...,...,...
80133,0.002586,0.380228,0.252874,0.078788,0.873239,0.198312,0.840694
80134,0.001799,0.733840,0.218391,0.024242,0.791549,0.451477,0.806309
80135,0.001349,0.730038,0.218391,0.109091,0.588732,0.449367,0.799369
80136,0.001461,0.730038,0.218391,0.078788,0.650704,0.447257,0.820347


In [23]:
df_y.shape

(80138, 19)

In [24]:
y_pred.shape

(40383, 8)